# *Lab 3 - Netflix Recommendation System*

In [1]:
# Extrating features from Wikipedia

In [1]:
import pandas as pd
import numpy as np

In [13]:
link = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2018'

In [19]:
df1 = pd.read_html(link)[2]
df2 = pd.read_html(link)[3]
df3 = pd.read_html(link)[4]
df4 = pd.read_html(link)[5]

In [27]:
df = pd.concat([df1,df2,df3,df4]).reset_index(drop=True)

In [36]:
import tmdbv3api
import json
import requests

tmdb = tmdbv3api.TMDb()

In [37]:
apiauth = '22396953bf97d94aad77322a4980e2d4'
tmdb.api_key = apiauth

In [38]:
from tmdbv3api import Movie

def get_genre(x):
    genres = []
    result = Movie().search(x)
    movie_id = result[0].id
    response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
    data_json = response.json()
    if data_json['genres']:
        for i in range(0, len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return ' '.join(genres)
    else:
        np.nan

In [42]:
# Extract genres

In [40]:
df['genres'] = df['Title'].apply(lambda x:get_genre(str(x)))

In [46]:
df2018 = df[['Title','Cast and crew','genres']].copy()

In [56]:
# Extract Director

In [110]:
def get_directors(x):
    if '(director)' in x:
        return x.split('(director); ')[0]
    elif '(director/screenplay)' in x:
        return x.split('(director/screenplay); ')[0]
    else:
        return x.split('(directors); ')[0]

In [111]:
df2018['directors'] = df2018['Cast and crew'].apply(lambda x:get_directors(str(x)))

In [57]:
# Extract Actor 1

In [113]:
def get_actor1(x):
    if 'screenplay)' in x:
        return x.split('screenplay); ')[-1].split(',')[0]

In [114]:
df2018['actor_1_name'] = df2018['Cast and crew'].apply(lambda x:get_actor1(str(x)))

In [ ]:
# Extract Actor 2

In [104]:
def get_actor2(x):
    if len(x.split('screenplay);')[-1].split(', '))< 2:
        return np.nan
    else:
        return x.split('screenplay)')[-1].split(',')[1]

In [105]:
df2018['actor_2_name'] = df2018['Cast and crew'].apply(lambda x:get_actor2(str(x)))

In [106]:
# Extract Actor 3

In [107]:
def get_actor3(x):
    if len(x.split('screenplay);')[-1].split(', '))< 3:
        return np.nan
    else:
        return x.split('screenplay)')[-1].split(',')[2]

In [108]:
df2018['actor_3_name'] = df2018['Cast and crew'].apply(lambda x:get_actor3(str(x)))

In [118]:
df2018.rename(columns={'Title':'movie_title'}, inplace=True)

In [121]:
df2018_ = df2018.drop('Cast and crew', axis=1).copy()

In [130]:
df2018_['actor_1_name'].replace(np.nan, 'unknown',inplace=True)

In [131]:
df2018_['actor_2_name'].replace(np.nan, 'unknown',inplace=True)

In [132]:
df2018_['actor_3_name'].replace(np.nan, 'unknown',inplace=True)

In [136]:
df2018_['movie_title'] = df2018_['movie_title'].str.lower()

In [141]:
df2018_.rename(columns={'directors':'director_name'}, inplace=True)

In [142]:
df2018_['Combined'] = df2018_['movie_title'] + df2018_['actor_1_name']+ df2018_['actor_2_name']+ df2018_['actor_3_name']+df2018_['director_name']+ df2018_['genres']

In [144]:
df2018_

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,Combined
0,insidious: the last key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,insidious: the last keyLin Shaye Angus Sampson...
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,the strange onesAlex Pettyfer James Freedson-J...
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,strattonDominic Cooper Austin Stowell Gemma Ch...
3,sweet country,Drama Western Thriller,Warwick Thornton,Bryan Brown,Sam Neill,unknown,sweet countryBryan Brown Sam NeillunknownWarwi...
4,the commuter,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,the commuterLiam Neeson Vera Farmiga Patrick W...
...,...,...,...,...,...,...,...
269,holmes & watson,Mystery Adventure Comedy Crime,Etan Cohen,Will Ferrell,John C. Reilly,Rebecca Hall,holmes & watsonWill Ferrell John C. Reilly Reb...
270,vice,Action Crime Thriller,Adam McKay,Christian Bale,Amy Adams,Steve Carell,viceChristian Bale Amy Adams Steve CarellAdam ...
271,on the basis of sex,Drama History,Mimi Leder,Felicity Jones,Armie Hammer,Justin Theroux,on the basis of sexFelicity Jones Armie Hammer...
272,destroyer,Documentary,Karyn Kusama,Nicole Kidman,Sebastian Stan,Toby Kebbell,destroyerNicole Kidman Sebastian Stan Toby Keb...


In [147]:
df2018_.to_csv('Processed/FinalData2.csv',index = False)